In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [11]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('elu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 28s 80ms/step - loss: 4.5670 - accuracy: 0.0363 - top3_acc: 0.0942 - top_k_categorical_accuracy: 0.1395 - val_loss: 3.9076 - val_accuracy: 0.1194 - val_top3_acc: 0.2569 - val_top_k_categorical_accuracy: 0.3453
Epoch 2/50
313/313 [==============================] - 23s 73ms/step - loss: 3.9242 - accuracy: 0.0969 - top3_acc: 0.2223 - top_k_categorical_accuracy: 0.3094 - val_loss: 3.5985 - val_accuracy: 0.1796 - val_top3_acc: 0.3390 - val_top_k_categorical_accuracy: 0.4360
Epoch 3/50
313/313 [==============================] - 26s 82ms/step - loss: 3.7096 - accuracy: 0.1379 - top3_acc: 0.2898 - top_k_categorical_accuracy: 0.3868 - val_loss: 3.5262 - val_accuracy: 0.1823 - val_top3_acc: 0.3435 - val_top_k_categorical_accuracy: 0.4479
Epoch 4/50
313/313 [==============================] - 25s 78ms/step - loss: 3.5560 - accuracy: 0.1716 - top3_acc: 0.3353 - top_k_categorical_accuracy: 0.4366 - val_loss: 3.5053 - val_accuracy: 0.1796 - val_to

In [12]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.309631824493408, 3.862114906311035, 3.6751246452331543, 3.5373785495758057, 3.4223129749298096, 3.3088085651397705, 3.213998794555664, 3.1210951805114746, 3.041996955871582, 2.96500825881958, 2.8970277309417725, 2.82362961769104, 2.7611002922058105, 2.6946194171905518, 2.6401352882385254, 2.5775132179260254, 2.5163888931274414, 2.455620765686035, 2.3963241577148438, 2.346095085144043, 2.2898170948028564, 2.244133949279785, 2.1884100437164307, 2.1330506801605225, 2.076795816421509, 2.0227270126342773, 1.9687306880950928, 1.9177875518798828, 1.863970160484314, 1.809874415397644, 1.7527943849563599, 1.709821105003357, 1.6619775295257568, 1.6136820316314697, 1.5573487281799316, 1.5046218633651733, 1.4579986333847046, 1.41689133644104, 1.362059235572815, 1.3192545175552368, 1.2730566263198853, 1.2311906814575195, 1.1804354190826416, 1.1377451419830322, 1.0890932083129883, 1.0481886863708496, 1.0044994354248047, 0.9628770351409912, 0.9168583154678345, 0.8883939385414124], 'accura